<a href="https://colab.research.google.com/github/PendlimarriSivasankar/PS/blob/main/Pyspark_Scenario_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("colab pyspark").getOrCreate()


In [4]:
from pyspark.sql.functions import lead, col, expr
from pyspark.sql.window import Window

data = [(1111, "2021-01-15", 10),
        (1111, "2021-01-16", 15),
        (1111, "2021-01-17", 30),
        (1112, "2021-01-15", 10),
        (1112, "2021-01-15", 20),
        (1112, "2021-01-15", 30)]

myschema = ["sensorid", "timestamp", "values"]

df = spark.createDataFrame(data, schema=myschema)
df.show()
d1 = Window.partitionBy("sensorid").orderBy("values")

finaldf = df.withColumn("nextvalues", lead("values", 1).over(d1)) \
    .filter(col("nextvalues").isNotNull()) \
    .withColumn("values", expr("nextvalues-values")) \
    .drop("nextvalues") \
    .orderBy(col("sensorid")).show()

+--------+----------+------+
|sensorid| timestamp|values|
+--------+----------+------+
|    1111|2021-01-15|    10|
|    1111|2021-01-16|    15|
|    1111|2021-01-17|    30|
|    1112|2021-01-15|    10|
|    1112|2021-01-15|    20|
|    1112|2021-01-15|    30|
+--------+----------+------+

+--------+----------+------+
|sensorid| timestamp|values|
+--------+----------+------+
|    1111|2021-01-15|     5|
|    1111|2021-01-16|    15|
|    1112|2021-01-15|    10|
|    1112|2021-01-15|    10|
+--------+----------+------+



In [7]:
##WITHCOLUMN 1


data="""{
    "id": 1,
    "institute": "zeyo",
    "trainer": "Sai",
    "zeyoAddress" :{
                        "permanentAddress" : "Hyderabad",
                        "temporaryAddress" : "chennai"
    }
}"""

df = spark.read.json(spark.sparkContext.parallelize([data]))

df.show()
df.printSchema()

flatdf = df.select(
    "id",
    "institute",
    "trainer",
    "zeyoAddress.permanentAddress",
    "zeyoAddress.temporaryAddress"
)

flatdf.show()
flatdf.printSchema()

withflatdf =(
    df.withColumn("permanentAddress",expr("zeyoAddress.permanentAddress"))
    .withColumn("temporaryAddress", expr("zeyoAddress.temporaryAddress"))
    .drop("zeyoAddress")
)

withflatdf.show()
withflatdf.printSchema()

+---+---------+-------+--------------------+
| id|institute|trainer|         zeyoAddress|
+---+---------+-------+--------------------+
|  1|     zeyo|    Sai|{Hyderabad, chennai}|
+---+---------+-------+--------------------+

root
 |-- id: long (nullable = true)
 |-- institute: string (nullable = true)
 |-- trainer: string (nullable = true)
 |-- zeyoAddress: struct (nullable = true)
 |    |-- permanentAddress: string (nullable = true)
 |    |-- temporaryAddress: string (nullable = true)

+---+---------+-------+----------------+----------------+
| id|institute|trainer|permanentAddress|temporaryAddress|
+---+---------+-------+----------------+----------------+
|  1|     zeyo|    Sai|       Hyderabad|         chennai|
+---+---------+-------+----------------+----------------+

root
 |-- id: long (nullable = true)
 |-- institute: string (nullable = true)
 |-- trainer: string (nullable = true)
 |-- permanentAddress: string (nullable = true)
 |-- temporaryAddress: string (nullable = true)

+-